In [1]:
import os
import gc
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import sys
sys.path.append( '../util' )
import util as util

/var/folders/1s/tyt3m2l12dz6s028_blqch7r0000gn/T/ipykernel_66854/1971013312.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# DNN with 2 classes

In [2]:

def create_binary_classification_model(input_shape):
    inputs = Input(shape=(input_shape,))
    x = Dense(128, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

train, test = util.import_dataset(2,"dnn")



y_train = train[util.y_column]
y_test = test[util.y_column]

X_train = train.drop(util.y_column, axis=1)
X_test = test.drop(util.y_column, axis=1)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

del train,test,y_train,y_test






model = create_binary_classification_model(len(util.X_columns))
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
#https://keras.io/api/models/model_training_apis/
history = model.fit(x=X_train, y=y_train_encoded,
                    validation_split=0.2, epochs=100, 
                    batch_size=256, callbacks=[early_stopping])


test_loss, test_acc = model.evaluate(X_test, y_test_encoded, verbose=2)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')
del X_train,X_test,y_train,y_test

Importing datasets.....
Processing training and test data....
Finished processing training and test data
Mapping labels to two classes.....
Data import and processing complete....
Epoch 1/100
113583/113583 ━━━━━━━━━━━━━━━━━━━━ 540s 5ms/step - accuracy: 0.9894 - loss: 0.0283 - val_accuracy: 0.9937 - val_loss: 0.0146
Epoch 2/100
113583/113583 ━━━━━━━━━━━━━━━━━━━━ 554s 5ms/step - accuracy: 0.9935 - loss: 0.0150 - val_accuracy: 0.9940 - val_loss: 0.0139
Epoch 3/100
113583/113583 ━━━━━━━━━━━━━━━━━━━━ 487s 4ms/step - accuracy: 0.9937 - loss: 0.0145 - val_accuracy: 0.9941 - val_loss: 0.0135
Epoch 4/100
113583/113583 ━━━━━━━━━━━━━━━━━━━━ 3796s 33ms/step - accuracy: 0.9938 - loss: 0.0142 - val_accuracy: 0.9942 - val_loss: 0.0135
Epoch 5/100
113583/113583 ━━━━━━━━━━━━━━━━━━━━ 1715s 15ms/step - accuracy: 0.9939 - loss: 0.0140 - val_accuracy: 0.9939 - val_loss: 0.0143
Epoch 6/100
113583/113583 ━━━━━━━━━━━━━━━━━━━━ 2614s 23ms/step - accuracy: 0.9939 - loss: 0.0139 - val_accuracy: 0.9943 - val_loss: